In [ ]:
import tensorflow as tf
import numpy as np

#parameter
train_file="data/book/train"
test_file="data/book/test"
dictionary_size=10000
stopword_size=100



In [ ]:
#step1
#reading data 
class OneEntry:
    def __init__(self,line):
        self.terms={}
        for feature in line.split(" "):
            feature_str=feature.split(":")
            self.terms[feature_str[0]]=feature_str[1]
        
        self.sentiment=self.terms["#label#"]
        del self.terms["#label#"]

def make_entries(filename):
    entries=[]
    with open(filename,"r") as f:
        for line in f.readlines():
            entries.append(OneEntry(line.rstrip()))
    
    return entries

entries=make_entries(train_file)
print len(entries),"entries"

#test to print one entry
print "sentiment",entries[0].sentiment
print entries[0].terms

In [ ]:
#step2
#making entire term dictionary
def get_whole_dictionary(entries):
    #merge all entries terms
    mergedterms={}
    for entry in entries:
        for term,freq in entry.terms.items():
            mergedterms[term]=mergedterms.get(term,0)+int(freq)
    
    return mergedterms

entire_terms=get_whole_dictionary(entries)
dictionary=sorted(entire_terms.items(),key=lambda x:x[1],reverse=True)[stopword_size:stopword_size+dictionary_size]
dictionary=[term[0] for term in dictionary]

#test print
print len(dictionary),"words"
print dictionary[0:10]

In [ ]:
#step3
#making batch data for learning
def get_batch(size,page,entries,dictionary):
    assert page>0
    assert len(entries) >= size * page
    
    x=np.zeros(shape=(size,len(dictionary)),dtype=np.int32)
    y=np.zeros([size,2])
    for datanum,entry in enumerate(entries[(page-1)*size:size*page]):
        for term,freq in entry.terms.items():
            if term in dictionary:
                x[datanum][dictionary.index(term)]+=int(freq)
        if float(entry.sentiment) > 3.0:
            y[datanum][1]=1
        else:
            y[datanum][0]=1
    
    return x,y

#making test batch
x_tmp,y_tmp=get_batch(8,1,entries,dictionary)
print x_tmp[0],y_tmp[1]


In [ ]:
#step4
#defining nural network 
print "dictionary_size",dictionary_size

#input feature
x=tf.placeholder(tf.float32,[None,dictionary_size])

#Ops
W=tf.Variable(tf.zeros([dictionary_size,2]))
b=tf.Variable(tf.zeros([2]))

#predictions
y=tf.nn.softmax(tf.matmul(x,W)+b)

#true label
y_=tf.placeholder(tf.float32,[None,2])

#learning process
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)



In [ ]:
#step5
#learning
batch_size=10
entries_len=len(entries)

init = tf.initialize_all_variables()
sess=tf.Session()
sess.run(init)
entropy=0

for step in range(entries_len/batch_size):
    batch_x,batch_y=get_batch(batch_size,step+1,entries,dictionary)
    sess.run(train_step,feed_dict={x:batch_x,y_:batch_y})
    
    if step%100:
        print "Average cross entropy",entropy/10
        entropy=0

In [ ]:
#step6
#evaluating
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

test_entries=make_entries(test_file)
test_x,test_y=get_batch(len(test_entries),1,test_entries,dictionary)
print "accuracy:",sess.run(accuracy, feed_dict={x: test_x, y_: test_y})